In [1]:
import gzip
import pickle
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, \
                                    Activation, Dropout, MaxPooling2D

def evaluate(path, model):
    X, y = pickle.load(gzip.open(path, 'rb'))
    y[y != 0] -= 2
    X = X / 255.
    acc = np.mean(model(X).numpy().argmax(axis=1) == y)
    return acc


In [2]:
path = 'flatland_train.data'
X,y = pickle.load(gzip.open(path,'rb')) 

# data pre-proc
y[y != 0] -= 2
X = X.reshape (X.shape[0],50,50,1) / 255.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 42)

DNN modelis

In [5]:
# model = keras.models.Sequential()
# model.add(keras.layers.Flatten(input_shape=[50, 50]))
# model.add(keras.layers.Dense(64, activation="relu"))
# model.add(keras.layers.Dense(32, activation="relu"))
# model.add(keras.layers.Dense(10, activation="softmax"))
# model.compile(loss="sparse_categorical_crossentropy",
#               optimizer="Adam",
#               metrics=["accuracy"])
# model.summary()

CNN modelis

In [4]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=[50, 50, 1]))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))
model.add(keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(keras.layers.BatchNormalization(axis=-1)) 
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2),strides=2))


model.add(Flatten())
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="Adam",
              metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        18496     
                                                                 
 batch_normalization_1 (Batc  (None, 22, 22, 64)       256       
 hNormalization)                                                 
                                                        

In [5]:
loss = model.fit(X_train, y_train, epochs=10,validation_data=[X_test,y_test])


Epoch 1/10
219/219 [==============================] - 5s 15ms/step - loss: 0.9042 - accuracy: 0.6517 - val_loss: 9.9969 - val_accuracy: 0.1520
Epoch 2/10
219/219 [==============================] - 3s 14ms/step - loss: 0.3013 - accuracy: 0.9136 - val_loss: 6.3567 - val_accuracy: 0.1520
Epoch 3/10
219/219 [==============================] - 3s 14ms/step - loss: 0.1745 - accuracy: 0.9611 - val_loss: 0.7458 - val_accuracy: 0.7067
Epoch 4/10
219/219 [==============================] - 3s 14ms/step - loss: 0.1348 - accuracy: 0.9703 - val_loss: 0.1788 - val_accuracy: 0.9700
Epoch 5/10
219/219 [==============================] - 3s 13ms/step - loss: 0.0990 - accuracy: 0.9761 - val_loss: 0.2136 - val_accuracy: 0.9587
Epoch 6/10
219/219 [==============================] - 3s 13ms/step - loss: 0.0907 - accuracy: 0.9734 - val_loss: 0.2451 - val_accuracy: 0.9563
Epoch 7/10
219/219 [==============================] - 3s 13ms/step - loss: 0.0913 - accuracy: 0.9757 - val_loss: 0.2416 - val_accuracy: 0.9480

In [7]:
model.save('model_cnn14.h5')


In [8]:
model = keras.models.load_model('model_cnn14.h5')
evaluate('flatland_train.data', model)

0.9856

In [6]:
pred = model.predict(X_test).argmax(axis=1)
print('Accuracy on test set - {0:.02%}'.format((pred == y_test).mean()))

Accuracy on test set - 97.50%
